In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import sys
from pathlib import Path
import datetime as dt
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import json


In [ ]:
RDBENCH_OUTPUT_DIR = Path("../raw/rdbench")

df_src = pl.DataFrame([
    {
        key: value
        for key, value in json.loads(json_file.read_text()).items()
        if key != "phaseDurationsSec"
    }
    for log_dir in RDBENCH_OUTPUT_DIR.glob("*")
    for json_file in log_dir.glob("rdbench_*.json")
])

df_src.head(5)

In [ ]:
df = df_src

fig = plt.figure(figsize=(16, 9), dpi=100)
fig.patch.set_alpha(1)
ax = (
    df
    .group_by("nprocs")
    .agg([
        pl.mean("writeBandwidthByte"),
    ])
    .sort("nprocs")
    .with_columns(bwMB=pl.col("writeBandwidthByte") / 10**6)
    .select(["nprocs", "bwMB"])
    .pipe(lambda df: (display(df), df)[-1])
    .to_pandas()
    .set_index("nprocs")
    .plot(
        ax=fig.add_subplot(),
        kind="bar",
        xlabel="Number of Procs",
        ylabel="Bandwidth [MB/s]",
        rot=0,
        width=0.7,
    )
)
ax.set_title("RDBench Write")